<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import sqlite3
from tabulate import tabulate

db = 'site.db'

def show_tables():
  tables = ['users', 'chats', 'messages']
  def show(tablename):
    with sqlite3.connect(db) as connection:
      cursor = connection.cursor()
      cursor.execute(f'SELECT * FROM {tablename}')
      results = cursor.fetchall()
      headers = []
      for i in range(len(cursor.description)):
        headers.append(cursor.description[i][0])
      print(tablename)
      print(tabulate(results, headers=headers, tablefmt='grid', stralign='center'))
      print("\n")

  for table in tables:
    show(table)

show_tables()

users
+------+----------+---------------------+
|   id |   name   |        email        |
+======+==========+=====================+
|    1 | John Doe | johndoe@example.com |
+------+----------+---------------------+


chats
+------+---------+-----------+
|   id |  name   |   user_id |
+======+=========+===========+
|    1 | General |         1 |
+------+---------+-----------+


messages
+------+-----------+-----------+-----------+--------------+
|  id  |  chat_id  |  user_id  |  message  |  created_at  |
+======+===========+===========+===========+==============+
+------+-----------+-----------+-----------+--------------+




In [137]:
def create_db():
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute("""CREATE TABLE users (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      name VARCHAR(255) NOT NULL,
                      email VARCHAR(255) NOT NULL
                    );""")

    cursor.execute("""CREATE TABLE chats (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      name VARCHAR(255) NOT NULL,
                      user_id INTEGER NOT NULL,
                      FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE
                    );""")

    cursor.execute("""CREATE TABLE messages (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      chat_id INTEGER NOT NULL,
                      user_id INTEGER NOT NULL,
                      message TEXT NOT NULL,
                      created_at DATETIME NOT NULL,
                      FOREIGN KEY (chat_id) REFERENCES chats (id) ON DELETE CASCADE,
                      FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE
                    );""")

    connection.commit()

# create_db()


def insert(message: str, username: str, chatname: str):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    # cursor.execute("""INSERT INTO users (name, email) VALUES ('John Doe', 'johndoe@example.com');""")
    # cursor.execute("""INSERT INTO chats (name, user_id) VALUES ('General', 1);""")
    cursor.execute(f"""INSERT INTO messages (chat_id, user_id, message, created_at)
                      SELECT chats.id, users.id,{message}, CURRENT_TIMESTAMP
                      FROM users
                      JOIN chats ON chats.user_id = users.id
                      WHERE users.name={username} AND chats.name={chatname};""")
    connection.commit()



# insert('"hello"', '"John Doe"', '"General"')


def delete(username: str, chatname: str):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute(f"""DELETE FROM messages
                      WHERE chat_id = (
                      SELECT chats.id
                      FROM chats
                      JOIN users ON chats.user_id = users.id
                      WHERE users.name={username} AND chats.name={chatname});""")
    connection.commit()

delete('"John Doe"', '"General"')